In [2]:
import os
import sys
import time

import numpy as np
seed = 7
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
import keras.backend as K

sys.path.insert(0, os.path.abspath('../../Evolutron'))
from evolutron.motifs import motif_extraction
from evolutron.tools import load_dataset, none2str, Handle, shape, get_args
from evolutron.templates import callback_templates as cb

os.chdir('..')
import nets

Using TensorFlow backend.


In [7]:
x_train, y_train = load_dataset(data_id='file', padded='True', codes=True, code_key='family',
                                infile='/data/uniprot_fam_g100_s100_train.csv')
x_test, y_test = load_dataset(data_id='file', padded='True', codes=True, code_key='family', max_aa=len(x_train[0]),
                              infile='/data/uniprot_fam_g100_s100_test.csv')
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Dataset size: 197360
Dataset size: 49340


In [8]:
rng_state = np.random.get_state()
np.random.shuffle(x_train)
np.random.set_state(rng_state)
np.random.shuffle(y_train)

In [9]:
input_shape = x_train[0].shape
output_dim = y_train.shape[1]

conv=1
fc=
filters=100
filter_length=25
optimizer = 'nadam'
rate=0.002
batch_size=256

In [10]:
conv_net = nets.build_cofam_model(input_shape,
                                  output_dim,
                                  n_conv_layers=conv,
                                  n_fc_layers=fc,
                                  filters=filters,
                                  filter_length=filter_length,
                                  optimizer=optimizer,
                                  lr=rate)

Neural Network has 319868 trainable parameters
  #  Name                   Shape       Parameters
---  ---------------------  --------  ------------
  0  aa_seq                 1410x20              0
  1  Conv1                  1410x100         50000
  2  batch_normalization_1  1410x100           400
  3  max_pooling1d_1        1x100                0
  4  flatten_1              100                  0
  5  FCEnc1                 100              10100
  6  FCEnc2                 100              10100
  7  Classifier             2468            249268


In [11]:
# Train 100 epochs with validation
epochs=100
callbacks = cb.standard(patience=20, reduce_factor=.5)
conv_net.fit(x_train, y_train,
             epochs=epochs,
             batch_size=batch_size,
             validation_data=(x_test, y_test),
             callbacks=callbacks,
             return_best_model=True)

Train on 197360 samples, validate on 49340 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[256,1,1410,100]
	 [[Node: Conv1/convolution/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Conv1/convolution/ExpandDims, Conv1/convolution/ExpandDims_1)]]
	 [[Node: Mean_3/_43 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1236_Mean_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Conv1/convolution/Conv2D', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/thras/Envs/py3Evolutron/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance(

In [14]:
handle = Handle(conv=conv, fc=fc, filter_length=filter_length,filters=filters,data_id='uniprot_fam_g100_s100',model='DeepCoFAM')
conv_net.save_train_history(handle)
conv_net.save(handle)

History saved to: models/uniprot_fam_g100_s100/100_25_48_3_1_DeepCoFAM_test.history
Model saved to: models/uniprot_fam_g100_s100/100_25_48_3_1_DeepCoFAM_test.model
